In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Printar os detalhjes das quantidades de usuários, filmes e quantas vezes esses filmes receberam avaliações
overall_stats = pd.read_csv('u.info', header=None)
print("Detalhes de usuários, filmes e avaliações:", list(overall_stats[0]))

In [ ]:
# O mesmo id para o item é o mesmo id do filme e a coluna é o id renomeada para o id do filme

column_names1 = ['Id_Usuario', 'Id_Filme', 'Avaliações', 'timestamp']
dataset = pd.read_csv('u.data', sep='\t', header=None, names=column_names1)
dataset.head()

In [ ]:
# Quantidade de avaliações por usuário
dataset['Id_Usuário'].value_counts()

In [ ]:
d = 'movie id | movie title | release date | video release date | IMDb URL | unknown | Action | Adventure | Animation | Children | Comedy | Crime | Documentary | Drama | Fantasy | Film-Noir | Horror | Musical | Mystery | Romance | Sci-Fi | Thriller | War | Western'
column_names2 = d.split(' | ')
column_names2

In [ ]:
items_dataset = pd.read_csv('u.Item', sep='|', header=None, names=column_names2, encoding='latin-1')
items_dataset

In [ ]:
# Pegando as informações dos títulos dos filmes
movie_dataset = items_dataset[['movie id', 'movie title']]
movie_dataset.rename(columns={'movie id':'Id_Filme', 'movie title': 'Título do Filme'}, inplace=True)
movie_dataset.head()

In [ ]:
merged_dataset = pd.merge(dataset, movie_dataset, how='inner', on='Id_Filme')
merged_dataset.head()

In [ ]:
merged_dataset.shape

In [ ]:
# Quantidade de avaliações dos filmes
numero_avaliacoes = merged_dataset.groupby('Titulo do Filme')['Avaliações'].count().reset_index()


In [ ]:
numero_avaliacoes

In [ ]:
# Juntando a tabela de filmes com os Ratings com a tabela de quantidade de ratings por filmes
numero_avaliacoes_final = merged_dataset.merge(numero_avaliacoes,on='Titulo do Filme')

In [ ]:
numero_avaliacoes_final

In [ ]:
# Filtrar apenas os filmes com pelo menos 20 avaliações
merged_dataset_final = numero_avaliacoes_final[numero_avaliacoes_final['Avaliações_y']>=20]
merged_dataset_final.shape

In [ ]:
merged_dataset_final.rename(columns={'Avaliações_x': 'Avaliações', 'Avaliações_y': 'Numero de avaliações'}, inplace=True)

In [ ]:
merged_dataset_final.head()

In [ ]:
# Descartando os valores duplicados
merged_dataset_final.drop_duplicates(['Id_Usuário', 'Titulo do filme'], inplace=True)
merged_dataset_final

In [ ]:
# Transpor usuáris em colunas
Filmes_pivot = merged_dataset_final.pivot_table(columns='Titulo do Filme', index='Id_Usuario', values='Avaliações')

In [ ]:
Filmes_pivot.head()

In [ ]:
# Substituindo os valores não avaliados com 0
Filmes_pivot = Filmes_pivot.fillna(0)

In [ ]:
Filmes_pivot.head()

In [ ]:
# Trabalhando com matrizes esparsa
from scipy.sparse import csr_matrix
Filmes_sparse = crs_matrix(Filmes_pivot)

In [3]:
def similar_users(user_id, matrix, k=5):
    # criar um dataframe somente para usuarios que deseja-se recomendar os filmes
    user = matrix[matrix.index == user_id]
    
    # criar um dataframe para os outros usuarios
    other_user = matrix[matrix.index != user_id]

    # calcular a similaridade do cosseno entre o usuario desejado e todos os outros
    similarities = cosine_similarity(user, other_user)[0].tolist()

    # criar um indice para esses usuarios
    indices - other_user.index.tolist()

    # criar um dicionario com os pares dos usuarios e os indices de similaridades
    index_similarity = dict(zip(indices, similarities))

    # ordenar por similaridade
    index_similarity_sorted = sorted(index_similarity.items(), key=operator.itemgetter(1))
    index_similarity_sorted.reverse()

    # colocar os usuarios mais similares no topo e retornar os 5 mais similares
    top_user_similarities = index_similarity_sorted[:k]
    user = [u[0] for u in top_user_similarities]

    return users

current_user = 1

#try it out
similar_users_indices = similar_users(current_user, Filmes_pivot)
print(similar_users_indices)

In [ ]:
def recomend_item(user_index, similar_users_indices, matrix, items=10):

    # carregando o vetor com os usuarios similares
    similar_users = matrix[matrix.index.isin(similar_users_indices)]

    # calculando a media
    similar_users = similar_users.mean(axis=0)

    # converter em um dataframe
    similar_users_df = pd.DataFrame(similar_users, columns=['média'])

    # carregando o vetor para o usuario que se deseja recomendar o filme
    user_df = matrix[matrix.index == user_index]
    # transpor o dataframe para filtrar
    user_df_transposed = user_df.transpose()
    # renomear a coluna para avaliações
    user_df_transposed.columns = ['Avaliações']
    # remover qualquer linha com avaliação 0
    user_df_transposed = user_df_transposed[user_df_transposed['Avaliações']==0]
    # gerar uma lista
    filmes_unvisited = user_df_transposed.index.tolist()

    # filtra a media
    similar_users_df_filtered = similar_users_df[similar_users_df.index.isin(filmes_unvisited)]
    # ordenar os filmes com melhores notas
    similar_user_df_ordered = similar_users_df_filtered.sort_values(by=['media'], ascending=false)
    # pegar os filmes com melhores notas
    top_n_filmes = similar_user_df_ordered(items)

    return top_n_filmes

# recomendar os filmes
df_recomend = recomend_item(1, similar_users_indices, Filmes_pivot)


In [ ]:
df_recomend